<a href="https://colab.research.google.com/github/JoshuaPaul-lasisi/Customer_segmentation/blob/main/Customer_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SUSTAINABLE ACHIEVERS Project 1: Customer Segmentation

# Project Overview

**Introduction:**
Welcome to our collaborative notebook! In this section, we'll provide an overview of our project and the goals of the EDA phase.

**Project Background:**
Our project aims to perform customer segmentation analysis using an E-Commerce data gotten from Kaggle. The dataset includes [brief description of dataset contents].

**Objective:**
The primary objective of our EDA phase is to gain a deeper understanding of the dataset and identify key insights that will inform our segmentation strategy. By exploring various aspects of the data, we aim to uncover patterns, trends, and relationships that can guide our segmentation approach.

**Team Collaboration:**
We have divided the EDA tasks among team members to ensure efficient collaboration and thorough analysis. Each team member is responsible for investigating specific aspects of the data and providing insights based on their findings.

**Notebook Structure:** 
- Data Loading and Cleaning
- Univariate Analysis
- Bivariate Analysis
- Categorical Variable Analysis
- Time Series Analysis
- Outlier Detection

**Instructions:**
- Each team member should focus on their assigned section and provide detailed analysis and visualizations.
- Feel free to collaborate with other team members if you need additional insights or assistance.
- Document your findings clearly and provide explanations for any decisions or observations made during the analysis.

**Expected Output:**
By the end of the EDA phase, we aim to have a comprehensive understanding of the dataset, including its characteristics, distributions, and relationships between variables. This will lay the foundation for our segmentation modeling efforts in the next phase of the project.


Let's dive into the analysis and uncover valuable insights together!


# Introduction

Necessary imports

In [54]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import seaborn as sns


...and the dataframe to be used


In [55]:
# Second Dataframe used in the kaggle we went through in our meeting
df = pd.read_csv('https://raw.githubusercontent.com/sheidheda/SusAc-ML-Files/main/cs_data2.csv', encoding="ISO-8859-1")
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


# Data Loading & Cleaning

This section of the notebook focuses on preparing the dataset for analysis by loading it into the environment and performing initial data cleaning steps. These steps are crucial to ensure that the data is in a usable format and free from any inconsistencies that could affect the analysis results. This section will address any missing values, outliers, and duplicates in the dataset, ensuring that the data is clean and ready for further analysis.

______________________________________
### Peeking under the hood of the data



We get a quick glance of the data. Why?

To check:
- How big the data is
- the kind of data in each column
- to check for possible missing values
- to check how much space our data takes

In [56]:
#checking the info for df
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


We now have answers to our questions:
- **Data size** -> 541,909 entries
- **Kind of data** -> the dataset contains a mix of data types
- **Possible missing values** -> Description and CustomerID have non-null counts less than the total number of entries. This indicates that there might be missing values in these columns
- **Space data takes** -> Memory usage is approximately 33.1+ MB


A more concise depiction of our dataset inspecting the dimensions of our dataset shows our dataset is **large** with 541,909 columns and 8 columns as seen below


In [57]:
df.shape

(541909, 8)

The **CustomerID** column should not be a float but a string since it is an identifier that could have leading zeros especially for older customers.

We will therefore convert it to string and remove the decimal point

In [58]:
df['CustomerID'] = df['CustomerID'].astype(str).str[:-2]

The **Quantity** ordered should not be less than zero for it to get into an invoice. But just in case, we will check it...

In [59]:
# to check whether quantity column has negative values
if (df['Quantity'].unique() <0).any():

  print("there is presence of Negative values")

there is presence of Negative values


This shows that we have **refunds** in our dataset. We will consider this is in our **segmentation**.

## Sorting out duplicates

The next step is to consider if we have any duplicates in the data.

For large data like this, we will employ the **.duplicate()** function which shows columns with repeated values for all columns.

In [60]:
# Identify rows with duplicate values across all columns in df2
duplicates = df[df.duplicated()]

# Count the number of duplicate rows
number_of_duplicates = len(duplicates)

# Print a message to inform the user about the results
print(f"The DataFrame df has {number_of_duplicates} duplicate rows.")

The DataFrame df has 5268 duplicate rows.


...to view them

In [61]:
duplicates

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
517,536409,21866,UNION JACK FLAG LUGGAGE TAG,1,12/1/2010 11:45,1.25,17908,United Kingdom
527,536409,22866,HAND WARMER SCOTTY DOG DESIGN,1,12/1/2010 11:45,2.10,17908,United Kingdom
537,536409,22900,SET 2 TEA TOWELS I LOVE LONDON,1,12/1/2010 11:45,2.95,17908,United Kingdom
539,536409,22111,SCOTTIE DOG HOT WATER BOTTLE,1,12/1/2010 11:45,4.95,17908,United Kingdom
555,536412,22327,ROUND SNACK BOXES SET OF 4 SKULLS,1,12/1/2010 11:49,2.95,17920,United Kingdom
...,...,...,...,...,...,...,...,...
541675,581538,22068,BLACK PIRATE TREASURE CHEST,1,12/9/2011 11:34,0.39,14446,United Kingdom
541689,581538,23318,BOX OF 6 MINI VINTAGE CRACKERS,1,12/9/2011 11:34,2.49,14446,United Kingdom
541692,581538,22992,REVOLVER WOODEN RULER,1,12/9/2011 11:34,1.95,14446,United Kingdom
541699,581538,22694,WICKER STAR,1,12/9/2011 11:34,2.10,14446,United Kingdom


Considering that they are redundancies, we will remove them **completely** from the dataframe.

In [62]:
# Drop the duplicates
df.drop_duplicates(inplace = True)

Our next course of action is to confirm if we have any missing values in our dataset.

To specify which columns have missing values...

##  Correcting missing values

In [63]:
# Check for missing values in df2
missing_values = df.isnull().sum()

# Calculate missing value percentages as a ratio (avoiding data type issues)
missing_percentages = (missing_values / len(df)) * 100

# Print a summary of missing values for each column
for col in df.columns:
  # Check if there are any missing values (avoiding unnecessary comparisons)
  if missing_values[col] > 0:
    print(f"Column '{col}' has {missing_values[col]} missing values ({missing_percentages[col]:.2f}%)")

Column 'Description' has 1454 missing values (0.27%)


We find that the **Description** and **CustomerID** columns have 1454 and 135037 missing values respectively.

Since CustomerID identifies the customer and Description identifies the product, let's see if there are cases where both product and customer are not identified

In [64]:
# columns with both description and CustomerID as null
no_id_desc = df[(df['Description'].isnull()) & (df['CustomerID'].isnull())]
no_id_desc

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country


We are **positive** that there are such columns and we also notice that they are the **exact same amount** as the amount of missing values in the Description column.

We can conclude that this is a subset of the rows with missing customer identifiers.

Another **observation** is that some of our unit prices are **zero**. Let's check how many...

In [65]:
no_id_desc['UnitPrice'].unique()

array([], dtype=float64)

**All of them!!!!**
Since there is not unit price, it means there is no sale.

When there is no sale, there is no customer.

This part of the data therefore adds no value to our **customer segmentation** then.

We will continue without them.

In [66]:
df_null = df[-df['Description'].isnull()]

Now we check how many null values we have left in the dataset

In [67]:
df_null.isnull().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

In [68]:
df_null['CustomerID'].fillna('Null', inplace = True)

df_null.isnull().sum()

<ipython-input-68-dd59f8bd1a28>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_null['CustomerID'].fillna('Null', inplace = True)


InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

## The finished work

In [69]:
df_cleaned = df_null

df_cleaned.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850,United Kingdom
